In [1]:
# Fetch the name of the active conda environment in the shell
# The `env` variable will be used later in the JN to
# ensure that "!" commands (which send commands to the shell)
# are run within the correct conda environment.
import os
import sys
env = os.path.basename(sys.executable.removesuffix("/bin/python"))
env

'isce3'

In [2]:
# update shell environment with sardem, the latest isce3, etc.
# !conda env update -n isce3 --file ../environment.yml > conda_install_log.txt
!conda env update -n {env} --file ../environment.yml

Channels:
 - conda-forge
Platform: linux-64
Solving environment: failed
Channels:
 - conda-forge
Platform: linux-64
Solving environment: failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package isce3-cuda-0.24.1-h0240f8b_2 requires isce3 0.24.1 *_cuda, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ isce3-cuda 0.24.1**  is installable and it requires
│  └─ isce3 0.24.1 *_cuda, which can be installed;
└─ isce3 0.22.1**  is not installable because it conflicts with any installable versions previously reported.



In [3]:
# import the MAAP package to handle queries
from maap.maap import MAAP
import os
import time

In [4]:
bbox = "-118.56784,34.11424,-117.28475,34.5186"  # Los Angeles National Forest

dateRange = '2006-01-01T00:00:00Z,2011-12-31T23:59:59Z'  # ALOS-1 was operational from 2006-2011

direction_filter = "ASCENDING"  # Must be "ASCENDING", "DESCENDING", or "NO_FILTER"

beam_modes = ("FBS", "FBD")  # Must be ("FBS", "FBD"), ("FBD",), or ("FBS", ). ISCE3 only supports FBS and FBD

# https://docs.maap-project.org/en/latest/system_reference_guide/share_data.html#Share-Data
# out_dir = '~/my-public-bucket/alos1_test/station_fire'
base_dir = '/projects/alos1-dist/station_fire'
in_dir = f"{base_dir}/input"
out_dir = f"{base_dir}/output"

for f in [base_dir, in_dir, out_dir]:
    os.makedirs(f, exist_ok=True)


In [5]:
# invoke the MAAP search client
maap = MAAP()

# generated from this EDSC search: https://ade.maap-project.org:30052/search/granules?p=C1206485320-ASF&pg[0][v]=f&pg[0][gsk]=-start_date&sb[0]=-118.56784%2C34.11424%2C-117.28475%2C34.5186&tl=1723592727.617!3!!&fpc0=Earth+Observation+Satellites&fpsc0=Advanced+Land+Observing+Satellite+%28ALOS%29&fps0=ALOS&fpb0=Space-based+Platforms&lat=33.933473284085615&long=-118.56941823504046&zoom=7
results = maap.searchGranule(cmr_host="cmr.earthdata.nasa.gov",
                             bounding_box=bbox,page_num="1",
                             concept_id="C1206485320-ASF",
                             sort_key="-start_date",
                             limit=1000,
                             temporal=dateRange
                            )

print(f"Total number of search results: {len(results)}")

Total number of search results: 338


In [6]:
# Filter search results

filtered_results = []
n_excluded_for_direction = 0
n_excluded_for_beammode = 0
n_excluded_for_frame_num = 0
n_excluded_for_path_num = 0

for result in results:
    keep = True
    tmp = result['Granule']['AdditionalAttributes']['AdditionalAttribute']
    for item in tmp:

        assert direction_filter in ("ASCENDING", "DESCENDING", "NO_FILTER")
        if item['Name'] == 'ASCENDING_DESCENDING':
            if direction_filter != "NO_FILTER":
                if item['Values']['Value'] != direction_filter:
                    keep = False
                    n_excluded_for_direction += 1

        assert set(beam_modes).issubset({"FBS", "FBD"})
        if item['Name'] == 'BEAM_MODE':
            if item['Values']['Value'] not in beam_modes:
                n_excluded_for_beammode += 1
                keep = False

        if item['Name'] == 'FRAME_NUMBER':
            # print(item['Values']['Value'])
            # frame 670 covers the Station Fire
            if item['Values']['Value'] != '670':
                n_excluded_for_frame_num += 1
                keep = False

        if item['Name'] == 'PATH_NUMBER':
            # print(item['Values']['Value'])
            # frame 670 covers the Station Fire
            if item['Values']['Value'] != '216':
                n_excluded_for_path_num += 1
                keep = False
        
    # All filters passed. Keep this result.
    if keep:
        filtered_results.append(result)

print(f"Number of search results before filtering: {len(results)}")
print(f"{n_excluded_for_direction=}")
print(f"{n_excluded_for_beammode=}")
print(f"{n_excluded_for_frame_num=}")
print(f"{n_excluded_for_path_num=}")
print(f"Number of search results after filtering: {len(filtered_results)}")


Number of search results before filtering: 338
n_excluded_for_direction=97
n_excluded_for_beammode=71
n_excluded_for_frame_num=245
n_excluded_for_path_num=270
Number of search results after filtering: 34


In [222]:
filtered_results[0]

{'concept-id': 'G1208697167-ASF',
 'collection-concept-id': 'C1206485320-ASF',
 'revision-id': '1',
 'format': 'application/echo10+xml',
 'Granule': {'{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': '',
  'GranuleUR': 'ALPSRP271200670-L1.0',
  'InsertTime': '2011-05-17T23:42:13Z',
  'LastUpdate': '2011-05-18T00:00:00Z',
  'Collection': {'DataSetId': 'ALOS_PALSAR_LEVEL1.0'},
  'DataGranule': {'SizeMBDataGranule': '454.15',
   'ProducerGranuleId': 'ALPSRP271200670',
   'DayNightFlag': 'UNSPECIFIED',
   'ProductionDateTime': '2011-05-18T00:00:00Z'},
  'Temporal': {'RangeDateTime': {'BeginningDateTime': '2011-02-26T06:18:00Z',
    'EndingDateTime': '2011-02-26T06:18:08Z'}},
  'Spatial': {'HorizontalSpatialDomain': {'Geometry': {'GPolygon': {'Boundary': {'Point': [{'PointLongitude': '-118.423',
         'PointLatitude': '33.861'},
        {'PointLongitude': '-118.531', 'PointLatitude': '34.365'},
        {'PointLongitude': '-117.827', 'PointLatitude': '34.468'},
     

In [7]:
# Import the ALOS1 to GCOV pipeline module
import importlib.util

module_path = '../src/alos1_2_gcov/alos1_to_gcov.py'
spec = importlib.util.spec_from_file_location("alos1_to_gcov", module_path)
a1_to_gcov = importlib.util.module_from_spec(spec)
spec.loader.exec_module(a1_to_gcov)


In [34]:
# For each frame ID, gather largest bbox (This will be used for the RSLC->GCOV processing step)

from dataclasses import dataclass

@dataclass
class QueryBbox:
    FAR_START_LAT: float
    FAR_START_LON: float

    NEAR_START_LAT: float
    NEAR_START_LON: float
    
    NEAR_END_LAT: float
    NEAR_END_LON: float
    
    FAR_END_LAT: float
    FAR_END_LON: float

    def get_bbox(self) -> a1_to_gcov.LatLonBbox:
        """
        Get lat/lon bbox for North America.

        TODO: make robust for all lat/lon bboxes in the world

        Returns
        -------
        left, bottom, right, top : float
            In lat/lon coordinates
        """
        top = max(self.FAR_START_LAT, self.NEAR_START_LAT, self.NEAR_END_LAT, self.FAR_END_LAT)
        bottom = min(self.FAR_START_LAT, self.NEAR_START_LAT, self.NEAR_END_LAT, self.FAR_END_LAT)

        left = max(self.FAR_START_LON, self.NEAR_START_LON, self.NEAR_END_LON, self.FAR_END_LON)
        right = min(self.FAR_START_LON, self.NEAR_START_LON, self.NEAR_END_LON, self.FAR_END_LON)

        return a1_to_gcov.LatLonBbox(left=left, bottom=bottom, right=right, top=top)


# Construct a list of the metadata names from the search results;
# These metadata contain the lat/lon bbox coordinates.
lat_lon_metadata_names = []
for dist in ['FAR', 'NEAR']:
    for time in ['START', 'END']:
        for coord in ['LAT', 'LON']:
            lat_lon_metadata_names.append(f"{dist}_{time}_{coord}")

union_of_bboxes = None

for result in filtered_results:
    granule_attr = result['Granule']['AdditionalAttributes']['AdditionalAttribute']

    # kwargs for building a Bbox instance for this granule
    kwargs:dict[str, float] = {}
    for item in granule_attr:
        name_of_attribute = item['Name']
        if name_of_attribute not in lat_lon_metadata_names:
            continue
            
        # Success! Store this metadata for use in a Bbox
        kwargs[name_of_attribute] = float(item['Values']['Value'])

    granule_bbox = QueryBbox(**kwargs).get_bbox()
    if union_of_bboxes is None:
        union_of_bboxes = granule_bbox
        print(granule_bbox)
    else:
        union_of_bboxes = a1_to_gcov.compute_union_bbox(
            ll1=union_of_bboxes,
            ll2=granule_bbox
        )



print(f"Union of bboxes from all granules: {union_of_bboxes}")


LatLonBbox(left=-117.722, bottom=33.861, right=-118.531, top=34.468)
Union of bboxes from all granules: LatLonBbox(left=-117.722, bottom=33.825, right=-118.61, top=34.483)


In [41]:
online_access_urls = []

for item in filtered_results:
    tmp = item['Granule']['OnlineAccessURLs']['OnlineAccessURL'][0]['URL']
    assert tmp.startswith("https://")
    online_access_urls.append(tmp)
print(online_access_urls)


['https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP271200670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP264490670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP257780670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP251070670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP237650670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP230940670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP224230670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP217520670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP210810670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP197390670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP190680670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP183970670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP170550670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP163840670-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP

## DPS Run

In [42]:
# Turn into a list of URLs, and submit to DPS

online_access_urls = []

for item in filtered_results:
    tmp = item['Granule']['OnlineAccessURLs']['OnlineAccessURL'][0]['URL']
    assert tmp.startswith("https://")
    online_access_urls.append(tmp)
# print(online_access_urls)

jobs = []
for item in online_access_urls[:1]:
    jobs.append(
        maap.submitJob(
            identifier="test_to_l0b",
            algo_id="sar2-d2",
            version="gcov_pipeline",
            username="niemoell",
            # queue="maap-dps-gpu_worker-16gb",
            queue="maap-dps-sandbox",
            in_file=item,
            bbox=union_of_bboxes.get_as_str(),
            in_type="alos1",  # Choose from: 'alos1', 'l0b', 'rslc',
            out_type="l0b",  # Choose from: 'l0b', 'rslc', 'gcov'
            gcov_posting="0.000833334"
        )
      )
    print("Input: ", item)
    print("Job ID: ", jobs[-1].id)


Input:  https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP271200670-L1.0.zip
Job ID:  7d4b78fd-b42a-45dd-a407-c0b70a8a4eff


In [29]:
# jobs[0].id
# jobs[0].retrieve_result()
jobs[0].outputs[0]  # https link, e.g. 'https://maap-ops-workspace.s3-website-us-west-2.amazonaws.com/dataset/triaged_job/sar2-d2/gcov_pipeline/2024/11/21/9d1055b0-c99c-483f-ac8a-f35505822a0a'
# jobs[0].outputs[1]  # s3 link, e.g. 's3://s3-us-west-2.amazonaws.com:80/maap-ops-workspace/dataset/triaged_job/sar2-d2/gcov_pipeline/2024/11/21/9d1055b0-c99c-483f-ac8a-f35505822a0a'
# jobs[0].outputs[3]  # Error traceback message

fox


AttributeError: 'str' object has no attribute 'outputs'

## ADE Testing Section

In [225]:
# Download the ALOS-1 L1.0 granules
# WARNING - only use for ADE testing! Not DPS!

import multiprocessing
from multiprocessing.pool import ThreadPool
import os
from itertools import repeat

num_to_test = len(filtered_results)
num_to_test = 1

# L0B_alos1 = "/projects/alos1-dist/input"
# os.makedirs(L0B_alos1, exist_ok=True)

num_available_cpu = multiprocessing.cpu_count()
print(f"There are {num_available_cpu} CPUs available.")

num_threads = min(num_to_test, 5)
print(f"Starting a ThreadPool of {num_threads} threads...")
pool = ThreadPool(num_threads)

def download_granule(maap_item, input_dir):    
    return maap_item.getData(input_dir)

start = time.time()
input_files = pool.starmap(download_granule, zip(filtered_results[:num_to_test], repeat(in_dir)))
print(f"Time to download {len(filtered_results)} ALOS-1 L1.0 granules: {time.time()-start} seconds")

There are 32 CPUs available.
Starting a ThreadPool of 1 threads...
Time to download 34 ALOS-1 L1.0 granules: 0.006104946136474609 seconds


In [38]:
# Unpack ALOS-1 zip file and repackage into NISAR L0B format
f_bbox = union_of_bboxes
print(type(f_bbox.left))
!SAR2D2_ENV=isce3 ./run.sh {input_files[0]} {f_bbox.left} {f_bbox.bottom} {f_bbox.right} {f_bbox.top} alos1 l0b 100


<class 'float'>
usage: alos1_to_gcov.py [-h] -i INPUT_FILE [--dem DEM_FILE] -o
                        OUTPUT_DIRECTORY --bbox LEFT BOTTOM RIGHT TOP
                        --in_type INPUT_PRODUCT_TYPE --out_type
                        OUTPUT_PRODUCT_TYPE [--posting GCOV_POSTING]
alos1_to_gcov.py: error: argument --bbox: invalid float value: '{f_bbox.left}'
ERROR conda.cli.main_run:execute(49): `conda run python /projects/alos1-dist/sar2-d2/src/alos1_2_gcov/alos1_to_gcov.py --in-file {input_files[0]} --bbox {f_bbox.left} {f_bbox.bottom} {f_bbox.right} {f_bbox.top} --out-dir /projects/alos1-dist/sar2-d2/nasa/output --in_type alos1 --out_type l0b --gcov_posting 100` failed. (See above for error)


In [ ]:
# Setup some variables

base_file_name = os.path.splitext(os.path.basename(input_files[0]))[0]
base_file_name = base_file_name.split(".")[0]
l0b_file = os.path.join(out_dir, f"L0B_{base_file_name}.h5")
rslc_file = os.path.join(out_dir, f"RSLC_{base_file_name}.h5")
gcov_file = os.path.join(out_dir, f"GCOV_{base_file_name}.h5")

In [101]:
# OPTIONAL -- Make dem.tif
# (This is handled automatically during the generation of RSLC and/or GCOV)
padded_bbox = a1_to_gcov.pad_bbox(union_of_bboxes, pad=1.0)
dem_file = os.path.join(out_dir, "dem.tif")
a1_to_gcov.make_dem(padded_bbox.get_as_str(), dem_file)


[11/18 06:29:35] [INFO dem.py] Bounds: -119.61 32.825 -116.722 35.483
[11/18 06:29:35] [INFO cop_dem.py] Creating /projects/alos1-dist/station_fire/output/dem.tif
[11/18 06:29:35] [INFO cop_dem.py] Fetching remote tiles...
[11/18 06:29:35] [INFO cop_dem.py] Running GDAL command:
[11/18 06:29:35] [INFO cop_dem.py] gdalwarp /vsicurl/https://raw.githubusercontent.com/scottstanie/sardem/master/sardem/data/cop_global.vrt /projects/alos1-dist/station_fire/output/dem.tif -of GTiff -ot Int16 -te -119.609999999999999 32.8250000000000028 -116.721999999999994 35.482999999999997 -tr 0.000277777777777777778 0.000277777777777777778 -s_srs "epsg:4326+3855" -t_srs "epsg:4326" -wo NUM_THREADS=4 -r nearest -wm 5000 -multi


Creating output file that is 10397P x 9569L.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [102]:
# Step 5: Focus L0B to RSLC

start = time.time()
!SAR2D2_ENV={env} ./run.sh {l0b_file} {f_bbox.left} {f_bbox.bottom} {f_bbox.right} {f_bbox.top} l0b rslc 100
print(f"Time to focus into RSLC: {time.time()-start} seconds")


In [212]:
# Step 5: Process RSLC to GCOV

# start = time.time()
# !./run.sh {rslc_file} {f_bbox.left} {f_bbox.bottom} {f_bbox.right} {f_bbox.top} rslc gcov 100
# print(f"Time to process RSLC into GCOV: {time.time()-start} seconds")

# Alternative to run the GCOV workflow from an existing runconfig file
# !python -m nisar.workflows.gcov /projects/alos1-dist/sar2-d2/nasa/output/gcov.yaml

# Use a sample NISAR product RSLC to test the GCOV workflow
# NISAR sample data
sample_bbox = a1_to_gcov.LatLonBbox(
    left=-118.480,
    bottom=34.235, 
    right=-117.473,
    top=35.364,
)
sample_rslc = "/projects/alos1-dist/R402_sample_products/NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001.h5"

!SAR2D2_ENV={env} ./run.sh {sample_rslc} {sample_bbox.left} {sample_bbox.bottom} {sample_bbox.right} {sample_bbox.top} rslc gcov 100


+ PIP_REQUIRE_VENV=0
+ /opt/conda/bin/conda env update --quiet --solver libmamba --name sar2-d2 --file /projects/alos1-dist/sar2-d2/environment.yml
+ PIP_REQUIRE_VENV=0
+ /opt/conda/bin/conda env update --quiet --solver libmamba --name sar2-d2 --file /projects/alos1-dist/sar2-d2/environment-dev.yml
[11/19 14:34:16] [INFO dem.py] Bounds: -119.48 33.235 -116.473 36.364
[11/19 14:34:16] [INFO cop_dem.py] Creating /projects/alos1-dist/sar2-d2/nasa/output/dem.tif
[11/19 14:34:16] [INFO cop_dem.py] Fetching remote tiles...
[11/19 14:34:16] [INFO cop_dem.py] Running GDAL command:
[11/19 14:34:16] [INFO cop_dem.py] gdalwarp /vsicurl/https://raw.githubusercontent.com/scottstanie/sardem/master/sardem/data/cop_global.vrt /projects/alos1-dist/sar2-d2/nasa/output/dem.tif -of GTiff -ot Int16 -te -119.480000000000004 33.2349999999999994 -116.472999999999999 36.3639999999999972 -tr 0.000277777777777777778 0.000277777777777777778 -s_srs "epsg:4326+3855" -t_srs "epsg:4326" -wo NUM_THREADS=4 -r nearest -